In [1]:
pip install numpy==1.26.4 torch transformers==4.48.2 scikit-learn accelerate==0.26.0 matplotlib tqdm pandas seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 229.6 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 146.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 50.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 234.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 373.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 73.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 422.8 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 221.8 kB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 206.0 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 265.8 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 158.4 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━

In [2]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
from tqdm import tqdm
import json

# =================== CONFIG ===================
DATA_DIR = "/workspace/SPLIT_SLIDING_FINAL/train"
OUTPUT_DIR = "/workspace/HASIL_ENCODER_Def/Hasil_2"
LABEL_MAP = {'AFIB': 0, 'VFL': 1, 'VT': 2}
N_SPLITS = 5
SEED = 42
EPOCHS = 20
BATCH_SIZE = 16
LR = 2e-5
MAX_LEN = 512
N_LAYERS = 12
D_MODEL = 768
N_HEAD = 12

torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =================== DATA LOADING ===================
def load_data(data_dir):
    data, labels = [], []
    for label_name in os.listdir(data_dir):
        if label_name not in LABEL_MAP:
            continue
        folder_path = os.path.join(data_dir, label_name)
        for file in os.listdir(folder_path):
            if file.endswith(".npy"):
                sig = np.load(os.path.join(folder_path, file), allow_pickle=True)
                if isinstance(sig, np.ndarray) and sig.ndim == 1:
                    data.append(sig.astype(np.float32))
                    labels.append(LABEL_MAP[label_name])
    return data, np.array(labels)

X, y = load_data(DATA_DIR)

# =================== SIGNAL TO TENSOR ===================
def signal_to_tensor(sig, target_len=MAX_LEN):
    if len(sig) < target_len:
        pad = np.full(target_len - len(sig), sig[-1])
        sig = np.concatenate([sig, pad])
    else:
        idx = np.linspace(0, len(sig) - 1, target_len).astype(int)
        sig = sig[idx]
    sig = (sig - sig.min()) / (sig.ptp() + 1e-8)
    return torch.tensor(sig, dtype=torch.float32)

X = [signal_to_tensor(sig) for sig in X]

# =================== DATASET ===================
class ECGDataset(Dataset):
    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.signals[idx], self.labels[idx]

# =================== MODEL ===================
class SimpleEncoder(nn.Module):
    def __init__(self, input_dim=MAX_LEN, d_model=D_MODEL, nhead=N_HEAD, num_layers=N_LAYERS, num_classes=3):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x.unsqueeze(1))
        x = self.encoder(x)
        x = x.mean(dim=1)
        return self.classifier(x)

# =================== METRICS ===================
def specificity_per_class(true, pred, label, num_classes):
    cm = confusion_matrix(true, pred, labels=list(range(num_classes)))
    TN = cm.sum() - (cm[label, :].sum() + cm[:, label].sum() - cm[label, label])
    FP = cm[:, label].sum() - cm[label, label]
    return TN / (TN + FP + 1e-8)

# =================== K-FOLD TRAINING ===================
all_results = []
best_f1 = 0
best_fold = 0

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n[INFO] Fold {fold}")

    train_set = ECGDataset([X[i] for i in train_idx], y[train_idx])
    val_set   = ECGDataset([X[i] for i in val_idx], y[val_idx])
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_set, batch_size=BATCH_SIZE)

    model = SimpleEncoder(num_classes=len(LABEL_MAP)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    history = []
    for epoch in range(1, EPOCHS+1):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for xb, yb in tqdm(train_loader, desc=f"Fold {fold} Epoch {epoch}"):
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = out.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = criterion(out, yb)
                val_loss += loss.item()
                preds = out.argmax(dim=1)
                val_correct += (preds == yb).sum().item()
                val_total += yb.size(0)

        val_loss = val_loss / len(val_loader)
        val_acc = val_correct / val_total

        print(f"[Epoch {epoch}] Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        history.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "train_acc": train_acc,
            "val_acc": val_acc
        })

    # Simpan riwayat training ke CSV
    fold_dir = os.path.join(OUTPUT_DIR, f"fold{fold}")
    os.makedirs(fold_dir, exist_ok=True)
    df_hist = pd.DataFrame(history)
    df_hist.to_csv(os.path.join(fold_dir, f"history_fold{fold}.csv"), index=False)

    # === Evaluation ===
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            out = model(xb)
            preds = out.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(yb.numpy())

    acc = accuracy_score(all_labels, all_preds)
    rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1s = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    cm = confusion_matrix(all_labels, all_preds)
    spec = np.mean([specificity_per_class(all_labels, all_preds, i, len(LABEL_MAP)) for i in range(len(LABEL_MAP))])

    torch.save(model.state_dict(), os.path.join(fold_dir, "encoder.pt"))
    torch.save(optimizer.state_dict(), os.path.join(fold_dir, "optimizer.pt"))
    torch.save(model, os.path.join(fold_dir, "encoder_full.pth"))

    config = {
        "input_dim": MAX_LEN,
        "d_model": D_MODEL,
        "nhead": N_HEAD,
        "num_layers": N_LAYERS,
        "num_classes": len(LABEL_MAP)
    }
    with open(os.path.join(fold_dir, "encoder_config.json"), "w") as f:
        json.dump(config, f, indent=4)

    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap="Blues", xticklabels=LABEL_MAP.keys(), yticklabels=LABEL_MAP.keys())
    plt.title(f"Confusion Matrix Fold {fold}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(os.path.join(fold_dir, "confmat_fold.png"))
    plt.close()

    df_per_class = []
    for idx, label in enumerate(LABEL_MAP):
        mask_true = (np.array(all_labels) == idx).astype(int)
        mask_pred = (np.array(all_preds) == idx).astype(int)
        acc_i = accuracy_score(mask_true, mask_pred)
        rec_i = recall_score(mask_true, mask_pred, zero_division=0)
        f1_i  = f1_score(mask_true, mask_pred, zero_division=0)
        spec_i = specificity_per_class(all_labels, all_preds, idx, len(LABEL_MAP))
        df_per_class.append([fold, label, acc_i, rec_i, f1_i, spec_i])

    df_fold = pd.DataFrame(df_per_class, columns=["fold", "kelas", "akurasi", "recall", "f1 score", "spesifisitas"])
    df_fold.to_csv(os.path.join(fold_dir, f"hasil_fold{fold}.csv"), index=False)

    all_results.append(df_fold)
    if f1s > best_f1:
        best_f1 = f1s
        best_fold = fold
        shutil.copyfile(os.path.join(fold_dir, "confmat_fold.png"), os.path.join(OUTPUT_DIR, "confmat_final.png"))
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "encoder_best.pt"))
        torch.save(model, os.path.join(OUTPUT_DIR, "encoder_best_full.pth"))
        with open(os.path.join(OUTPUT_DIR, "encoder_best_config.json"), "w") as f:
            json.dump(config, f, indent=4)

final_df = pd.concat(all_results, ignore_index=True)
final_df.to_csv(os.path.join(OUTPUT_DIR, "hasil_kfold_encoder.csv"), index=False)
print(f"[INFO] Fold terbaik berdasarkan F1-score: Fold {best_fold}")


[INFO] Fold 1


Fold 1 Epoch 1: 100%|██████████| 240/240 [00:05<00:00, 45.92it/s]


[Epoch 1] Train Loss: 0.3950, Val Loss: 0.1998, Train Acc: 0.8490, Val Acc: 0.9490


Fold 1 Epoch 2: 100%|██████████| 240/240 [00:04<00:00, 50.15it/s]


[Epoch 2] Train Loss: 0.1557, Val Loss: 0.1782, Train Acc: 0.9513, Val Acc: 0.9479


...

[Epoch 18] Train Loss: 0.0466, Val Loss: 0.1195, Train Acc: 0.9857, Val Acc: 0.9740


Fold 5 Epoch 19: 100%|██████████| 240/240 [00:04<00:00, 49.35it/s]


[Epoch 19] Train Loss: 0.0438, Val Loss: 0.1086, Train Acc: 0.9862, Val Acc: 0.9740


Fold 5 Epoch 20: 100%|██████████| 240/240 [00:04<00:00, 49.49it/s]


[Epoch 20] Train Loss: 0.0342, Val Loss: 0.1210, Train Acc: 0.9893, Val Acc: 0.9667
[INFO] Fold terbaik berdasarkan F1-score: Fold 1


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
from tqdm import tqdm
import json

# =================== CONFIG ===================
DATA_DIR = "/workspace/SPLIT_SLIDING_FINAL/train"
OUTPUT_DIR = "/workspace/HASIL_ENCODER_Tuning_Rhythm/Hasil_1"
LABEL_MAP = {'N': 0, 'AFIB': 1, 'VFL': 2}
SEED = 42
EPOCHS = 20
BATCH_SIZE = 16
MAX_LEN = 512

# === Parameter hasil tuning manual ===
D_MODEL = 512
N_HEAD = 8
LR = 2e-5
WEIGHT_DECAY = 1e-4
DROPOUT = 0.2
N_SPLITS = 5

# =================== UTILITIES ===================
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =================== DATA LOADING ===================
def load_data(data_dir):
    data, labels = [], []
    for label_name in os.listdir(data_dir):
        if label_name not in LABEL_MAP:
            continue
        folder_path = os.path.join(data_dir, label_name)
        for file in os.listdir(folder_path):
            if file.endswith(".npy"):
                sig = np.load(os.path.join(folder_path, file), allow_pickle=True)
                if isinstance(sig, np.ndarray) and sig.ndim == 1:
                    data.append(sig.astype(np.float32))
                    labels.append(LABEL_MAP[label_name])
    return data, np.array(labels)

X, y = load_data(DATA_DIR)

def signal_to_tensor(sig, target_len=MAX_LEN):
    if len(sig) < target_len:
        pad = np.full(target_len - len(sig), sig[-1])
        sig = np.concatenate([sig, pad])
    else:
        idx = np.linspace(0, len(sig) - 1, target_len).astype(int)
        sig = sig[idx]
    sig = (sig - sig.min()) / (sig.ptp() + 1e-8)
    return torch.tensor(sig, dtype=torch.float32)

X = [signal_to_tensor(sig) for sig in X]

class ECGDataset(Dataset):
    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return self.signals[idx], self.labels[idx]

class SimpleEncoder(nn.Module):
    def __init__(self, input_dim=MAX_LEN, d_model=768, nhead=12, num_layers=12, num_classes=3, dropout=0.1):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.classifier = nn.Linear(d_model, num_classes)
    def forward(self, x):
        x = self.embedding(x.unsqueeze(1))
        x = self.encoder(x)
        x = x.mean(dim=1)
        return self.classifier(x)

def specificity_per_class(true, pred, label, num_classes):
    cm = confusion_matrix(true, pred, labels=list(range(num_classes)))
    TN = cm.sum() - (cm[label, :].sum() + cm[:, label].sum() - cm[label, label])
    FP = cm[:, label].sum() - cm[label, label]
    return TN / (TN + FP + 1e-8)

all_results = []
best_f1 = 0
best_fold = 0

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\n[INFO] Fold {fold}")

    train_set = ECGDataset([X[i] for i in train_idx], y[train_idx])
    val_set   = ECGDataset([X[i] for i in val_idx], y[val_idx])
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_set, batch_size=BATCH_SIZE)

    model = SimpleEncoder(d_model=D_MODEL, nhead=N_HEAD, num_layers=12, num_classes=len(LABEL_MAP), dropout=DROPOUT).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

    history = []
    for epoch in range(1, EPOCHS+1):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for xb, yb in tqdm(train_loader, desc=f"Fold {fold} Epoch {epoch}"):
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = out.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = criterion(out, yb)
                val_loss += loss.item()
                preds = out.argmax(dim=1)
                val_correct += (preds == yb).sum().item()
                val_total += yb.size(0)

        val_loss = val_loss / len(val_loader)
        val_acc = val_correct / val_total

        print(f"[Epoch {epoch}] Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        history.append({
            "epoch": epoch,
            "train_loss": train_loss,
            "val_loss": val_loss,
            "train_acc": train_acc,
            "val_acc": val_acc
        })

    all_preds, all_labels = [], []
    model.eval()
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            out = model(xb)
            preds = out.argmax(dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(yb.numpy())

    f1s = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    spec = np.mean([specificity_per_class(all_labels, all_preds, i, len(LABEL_MAP)) for i in range(len(LABEL_MAP))])

    df_hist = pd.DataFrame(history)
    fold_dir = os.path.join(OUTPUT_DIR, f"fold{fold}")
    os.makedirs(fold_dir, exist_ok=True)
    df_hist.to_csv(os.path.join(fold_dir, f"history_fold{fold}.csv"), index=False)

    plt.figure(figsize=(6, 5))
    sns.heatmap(confusion_matrix(all_labels, all_preds), annot=True, fmt='d', cmap="Blues",
                xticklabels=LABEL_MAP.keys(), yticklabels=LABEL_MAP.keys())
    plt.title(f"Confusion Matrix Fold {fold}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(os.path.join(fold_dir, "confmat_fold.png"))
    plt.close()

    df_per_class = []
    for idx, label in enumerate(LABEL_MAP):
        mask_true = (np.array(all_labels) == idx).astype(int)
        mask_pred = (np.array(all_preds) == idx).astype(int)
        acc_i = accuracy_score(mask_true, mask_pred)
        rec_i = recall_score(mask_true, mask_pred, zero_division=0)
        f1_i  = f1_score(mask_true, mask_pred, zero_division=0)
        spec_i = specificity_per_class(all_labels, all_preds, idx, len(LABEL_MAP))
        df_per_class.append([fold, label, acc_i, rec_i, f1_i, spec_i])

    df_fold = pd.DataFrame(df_per_class, columns=["fold", "kelas", "akurasi", "recall", "f1 score", "spesifisitas"])
    df_fold.to_csv(os.path.join(fold_dir, f"hasil_fold{fold}.csv"), index=False)

    all_results.append(df_fold)
    if f1s > best_f1:
        best_f1 = f1s
        best_fold = fold
        shutil.copyfile(os.path.join(fold_dir, "confmat_fold.png"), os.path.join(OUTPUT_DIR, "confmat_final.png"))
        torch.save(model.state_dict(), os.path.join(OUTPUT_DIR, "encoder_best.pt"))
        torch.save(model, os.path.join(OUTPUT_DIR, "encoder_best_full.pth"))
        with open(os.path.join(OUTPUT_DIR, "encoder_best_config.json"), "w") as f:
            json.dump({"D_MODEL": D_MODEL, "N_HEAD": N_HEAD, "LR": LR, "DROPOUT": DROPOUT, "WEIGHT_DECAY": WEIGHT_DECAY, "MAX_LEN": MAX_LEN}, f, indent=4)

final_df = pd.concat(all_results, ignore_index=True)
final_df.to_csv(os.path.join(OUTPUT_DIR, "hasil_kfold_encoder.csv"), index=False)
print(f"[INFO] Fold terbaik berdasarkan F1-score: Fold {best_fold}")


[INFO] Fold 1


Fold 1 Epoch 1: 100%|██████████| 240/240 [00:04<00:00, 53.90it/s]


[Epoch 1] Train Loss: 0.4081, Val Loss: 0.2168, Train Acc: 0.8352, Val Acc: 0.9365


Fold 1 Epoch 2: 100%|██████████| 240/240 [00:04<00:00, 53.96it/s]


[Epoch 2] Train Loss: 0.2038, Val Loss: 0.1866, Train Acc: 0.9365, Val Acc: 0.9521


...

[Epoch 18] Train Loss: 0.0593, Val Loss: 0.1288, Train Acc: 0.9781, Val Acc: 0.9740


Fold 5 Epoch 19: 100%|██████████| 240/240 [00:05<00:00, 47.95it/s]


[Epoch 19] Train Loss: 0.0504, Val Loss: 0.1257, Train Acc: 0.9815, Val Acc: 0.9719


Fold 5 Epoch 20: 100%|██████████| 240/240 [00:05<00:00, 45.85it/s]


[Epoch 20] Train Loss: 0.0506, Val Loss: 0.1314, Train Acc: 0.9815, Val Acc: 0.9667
[INFO] Fold terbaik berdasarkan F1-score: Fold 3


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# ======== KONFIGURASI ========
TEST_DIR = "/workspace/SPLIT_SLIDING_FINAL/test"
MODEL_PATH = "/workspace/HASIL_ENCODER_Tuning_Rhythm/Hasil_1/encoder_best_full.pth"
OUTPUT_DIR = "/workspace/HASIL_ENCODER_Tuning_Rhythm/EVALUASI_TEST"
LABEL_MAP = {'N': 0, 'AFIB': 1, 'VFL': 2}
MAX_LEN = 512
BATCH_SIZE = 16

os.makedirs(OUTPUT_DIR, exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ======== DATA LOADING ========
def load_data(data_dir):
    data, labels = [], []
    for label_name in os.listdir(data_dir):
        if label_name not in LABEL_MAP:
            continue
        folder_path = os.path.join(data_dir, label_name)
        for file in os.listdir(folder_path):
            if file.endswith(".npy"):
                sig = np.load(os.path.join(folder_path, file), allow_pickle=True)
                if isinstance(sig, np.ndarray) and sig.ndim == 1:
                    data.append(sig.astype(np.float32))
                    labels.append(LABEL_MAP[label_name])
    return data, np.array(labels)

def signal_to_tensor(sig, target_len=MAX_LEN):
    if len(sig) < target_len:
        pad = np.full(target_len - len(sig), sig[-1])
        sig = np.concatenate([sig, pad])
    else:
        idx = np.linspace(0, len(sig) - 1, target_len).astype(int)
        sig = sig[idx]
    sig = (sig - sig.min()) / (sig.ptp() + 1e-8)
    return torch.tensor(sig, dtype=torch.float32)

class ECGDataset(Dataset):
    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return self.signals[idx], self.labels[idx]

# ======== SPESIFISITAS ========
def specificity_per_class(true, pred, label, num_classes):
    cm = confusion_matrix(true, pred, labels=list(range(num_classes)))
    TN = cm.sum() - (cm[label, :].sum() + cm[:, label].sum() - cm[label, label])
    FP = cm[:, label].sum() - cm[label, label]
    return TN / (TN + FP + 1e-8)

# ======== INFERENSI TEST ========
print("[INFO] Memuat data test...")
X_raw, y = load_data(TEST_DIR)
X = [signal_to_tensor(sig) for sig in X_raw]
test_dataset = ECGDataset(X, y)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print("[INFO] Memuat model terlatih...")
model = torch.load(MODEL_PATH)
model.to(device)
model.eval()

all_preds, all_labels = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        out = model(xb)
        preds = out.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(yb.numpy())

# ======== EVALUASI =========
cm = confusion_matrix(all_labels, all_preds)
acc = accuracy_score(all_labels, all_preds)
rec = recall_score(all_labels, all_preds, average='macro', zero_division=0)
f1s = f1_score(all_labels, all_preds, average='macro', zero_division=0)
spec = np.mean([specificity_per_class(all_labels, all_preds, i, len(LABEL_MAP)) for i in range(len(LABEL_MAP))])

print(f"[HASIL] Akurasi: {acc:.4f} | Recall: {rec:.4f} | Spesifisitas: {spec:.4f} | F1-Score: {f1s:.4f}")

# ======== SIMPAN HASIL ========
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=LABEL_MAP.keys(), yticklabels=LABEL_MAP.keys())
plt.title("Confusion Matrix - Test Data")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "confmat_test.png"))
plt.close()

# Perkelas
df_per_class = []
for idx, label in enumerate(LABEL_MAP):
    mask_true = (np.array(all_labels) == idx).astype(int)
    mask_pred = (np.array(all_preds) == idx).astype(int)
    acc_i = accuracy_score(mask_true, mask_pred)
    rec_i = recall_score(mask_true, mask_pred, zero_division=0)
    f1_i = f1_score(mask_true, mask_pred, zero_division=0)
    spec_i = specificity_per_class(all_labels, all_preds, idx, len(LABEL_MAP))
    df_per_class.append([label, acc_i, rec_i, f1_i, spec_i])

df_result = pd.DataFrame(df_per_class, columns=["kelas", "akurasi", "recall", "f1 score", "spesifisitas"])
df_result.to_csv(os.path.join(OUTPUT_DIR, "evaluasi_test_perkelas.csv"), index=False)

# Rata-rata
df_avg = pd.DataFrame([{
    "akurasi": acc,
    "recall": rec,
    "f1 score": f1s,
    "spesifisitas": spec
}])
df_avg.to_csv(os.path.join(OUTPUT_DIR, "evaluasi_test_ratarata.csv"), index=False)

[INFO] Memuat data test...
[INFO] Memuat model terlatih...
[HASIL] Akurasi: 0.9650 | Recall: 0.9650 | Spesifisitas: 0.9825 | F1-Score: 0.9652
